In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import py_vollib_vectorized as vollib
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from alpaca_trade_api.rest import REST

TRADIER_API_KEY = "Jta3vCKbfk5HxsN8hHqEp4HlyPku"
ALPACA_KEY_ID = 
ALPACA_SECRET_KEY = 
headers = {'Authorization': "Bearer {}".format(TRADIER_API_KEY), 'Accept': 'application/json'}
BASE_URL = "https://sandbox.tradier.com"

# building a screener to look for highly shorted stocks trading at below fair value, looking for opportunity to run up through potential meme interest from WSB: 
# 1. screen for stocks with high short interest
# 2. determine short interest is not consequence of a run-up and that stock is definitively below fair value
# 3. check implied volatility of long term calls and change in open interest, with highly shorted stocks we should expect to see 
# it hedged with calls or sold puts. Any data to the contrary suggests opportunity
#4. 

In [ ]:

def get_historical_data_for_security(ticker,start=str((datetime.today() - timedelta(days=365)).date()),end=str(datetime.today().date()),api_key = POLYGON_API_KEY,df=True):
    response = requests.get(f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start}/{end}?unadjusted=false&sort=asc&apiKey={api_key}")
    data = response.json()['results']
    if df: 
        data = pd.DataFrame(data)
        data['t'] = pd.to_datetime(data['t'],unit='ms')
        data['200MA'] = data['c'].rolling(200).mean()
        return data
    return data
    
def get_first_val_in_table(table):
    first_row = table.find('tr')
    if first_row:
        first_field = first_row.find('td')
        if first_field:
            enclosed = first_field.find('div')
            if enclosed:
                return enclosed.text
    return ''

def get_si_data_from_table(table,df=True):
    data = []
    cols = []
    rows = table.findAll('tr')
    for row in rows:
        col,val = row.findAll('td')
        cols.append(col.find('div').text.strip())
        data.append(val.text.strip())
    if df: return pd.DataFrame([data[2:]],columns=cols[2:]) # first 2 fields are bs
    return cols,data
        

def get_short_interest_for_security(ticker):
    response = requests.get("https://shortsqueeze.com/?symbol={}&submit=Short+Quote%E2%84%A2".format(ticker))
    soup = BeautifulSoup(response.text,'html.parser')
    si_table = [table for table in soup.findAll('table') if get_first_val_in_table(table) == 'Short Squeeze Ranking™'][0]
    return get_si_data_from_table(si_table)


def get_options_for_security(stock):
    response = requests.get('{}/v1/markets/options/lookup'.format(BASE_URL),
        params={'underlying': stock},
        headers=headers
    )

    json_response = response.json()
    return json_response['symbols'][0]['options']


def get_quotes_for_options(options,batch_size=25):
    chains = []
    for i in range(0, len(options),batch_size):
        options_batch = options[i:i+batch_size]
        response = requests.get('{}/v1/markets/quotes'.format(BASE_URL),
            params={'symbols': ','.join(options_batch), 'greeks': 'true'},
            headers=headers
        )    
        json_response = response.json()
        chains += json_response['quotes']['quote']
    return chains
    
def get_quote_for_security(stock):
    response = requests.get('{}/v1/markets/quotes'.format(BASE_URL),
        params={'symbols': stock, 'greeks': 'false'},
        headers=headers
    )    
    json_response = response.json()
    return json_response['quotes']['quote']
    

def get_most_shorted_stocks(df=True):
    table_cols = ['Symbol\nSymbol',
                 'Company Name',
                 'Price',
                 'Chg% (1D)',
                 'Chg% (YTD)',
                 'Short Interest',
                 'Short Date',
                 'Float',
                 'Float Shorted (%)']
    url = "https://www.marketwatch.com/tools/screener/short-interest"
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    si_table = soup.findAll('table')[0]
    si_cols =  [col.text.strip('\n') for col in si_table.find('thead').find('tr').findAll('th')]
    assert(si_cols == table_cols)
    rows = si_table.findAll('tr')[1:]
    data = []
    for row in rows: data.append([val.text.strip('\n') for val in row.findAll('td')])
    if df: return pd.DataFrame(data,columns=si_cols)
    return data


In [ ]:
data = get_historical_data_for_security('BLNK')
import time
def get_max_runup(df):
    gap = df['gap'] = (df['c'] - df['200MA'])*100/df['c']
    return df['gap'].max()

non_runup_stocks = ['BLNK','CLOV','GME','AMC','NKLA']
for stock in non_runup_stocks:
    data = get_historical_data_for_security(stock)
    max_runup = get_max_runup(data)
    print(f"{stock} max runup: {max_runup} %")
    time.sleep(20)


'Jta3vCKbfk5HxsN8hHqEp4HlyPku'